In [3]:
import openai
import os
import pyaudio
import wave
import numpy as np
import webrtcvad


def chat(model, api_key):
    # Authenticate with OpenAI
    openai.api_key = api_key
    
    # Create a list of dictionaries to store the conversation history
    conversation = [{"role": "system", "content": "You are a helpful assistant."}]
    
    # Loop through the conversation
    while True:
        
        # Get user input as recording
        CHUNK = 160
        FORMAT = pyaudio.paInt16
        CHANNELS = 2
        RATE = 16000
        SILENCE_LIMIT = 2  # Time of silence needed to stop recording (seconds)
        WAVE_OUTPUT_FILENAME = "input.wav"

        vad = webrtcvad.Vad()
        vad.set_mode(3)
    
        audio = pyaudio.PyAudio()

        stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

        print("Recording...")

        frames = []
        silent_count = 0
    
        while True:
            data = stream.read(CHUNK)
            
            #Check for speech
            decoded_data = np.frombuffer(data, dtype=np.int16)
            audio_level = np.max(decoded_data)

            if  vad.is_speech(data, RATE):
                silent_count = 0
                
                frames.append(data)
            else:
                silent_count += 1
            
            # Check if there's been enough silence to stop recording
            if silent_count > SILENCE_LIMIT * RATE / CHUNK:
                break

        print("Finished recording.")

        stream.stop_stream()
        stream.close()
        audio.terminate()
 
        waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(frames))
        waveFile.close()
        
        #Utilize Whisper for transcription
        audio_file= open("input.wav", "rb")
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
        
        #feed transript into conversation
        user_input = transcript["text"]
        conversation.append(
        {"role": "user", "content": user_input}
        )
        
        # Generate the model responses
        responses = openai.ChatCompletion.create(
            model=model,
            messages=conversation
        )
        
        # save the models response
        prime_response = responses.choices[0].message["content"]
        conversation.append({"role": "assistant", "content":prime_response}
        )
        
        # Print the model's response
        print(transcript["text"])
        print(responses.choices[0].message["content"])
        

chat(model="gpt-4", api_key=os.environ["OPENAI_API_KEY"])

Recording...
Finished recording.
Hello, JetJPT. I want you to create a contract for me. The contract will be by and between me and my cousin to attain our goals in exercise. Please write a contract in which it is agreed upon that we will do a weekly exercise of a total of 150 minutes. Make it simple, pragmatic and place it under the German jurisdiction. The contract will go on for one year. Leave out blanks to fill the dates, please.
**EXERCISE CONTRACT**

This Agreement ("Contract") is made this ____ day of _______, 20____, by and between __________("First Party") and ___________("Second Party"). Both parties herein are also referred herein as ("Party") or collectively as the ("Parties").

**I. AGREEMENT:**

1. The Parties hereby confirm and agree to engage in a minimum total of One Hundred Fifty (150) minutes of exercise every week for the duration of One (1) year effective from the date added on top of this Agreement.

**II. DURATION:**

1. This Contract is to commence on the date a

InvalidRequestError: Audio file is too short. Minimum audio length is 0.1 seconds.